# Thesis Experiment (Part A + Part B MVP)

Minimal notebook to run the thesis experiment end-to-end using existing project functions from `src/sharpe_mc.py`.

Outputs saved under `outputs/thesis_mvp/`:
- `results_partA.parquet` (and CSV fallback)
- `results_partB_pit.parquet` (and CSV fallback)
- `model_fit_summary.json`
- `environment_versions.json`
- figures (`.png` and `.pdf`)

In [21]:
import json
import platform
import random
import sys
from pathlib import Path
import importlib.metadata as ilmd

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs):
        return x

pd.set_option("display.max_columns", 200)
plt.style.use("ggplot")

RUN_DIR = Path("outputs/thesis_mvp").resolve()
RUN_DIR.mkdir(parents=True, exist_ok=True)
SAVED_ARTIFACTS = []


def pkg_version(name: str) -> str:
    try:
        return ilmd.version(name)
    except Exception:
        return "not-installed"


versions = {
    "python": sys.version,
    "platform": platform.platform(),
    "packages": {
        "numpy": pkg_version("numpy"),
        "pandas": pkg_version("pandas"),
        "scipy": pkg_version("scipy"),
        "matplotlib": pkg_version("matplotlib"),
        "tqdm": pkg_version("tqdm"),
        "arch": pkg_version("arch"),
    },
}
print(json.dumps(versions, indent=2))

versions_path = RUN_DIR / "environment_versions.json"
versions_path.write_text(json.dumps(versions, indent=2), encoding="utf-8")
SAVED_ARTIFACTS.append(str(versions_path))

{
  "python": "3.12.1 (main, Jul 10 2025, 11:57:50) [GCC 13.3.0]",
  "platform": "Linux-6.8.0-1030-azure-x86_64-with-glibc2.39",
  "packages": {
    "numpy": "2.3.4",
    "pandas": "2.3.3",
    "scipy": "1.16.3",
    "matplotlib": "3.10.3",
    "tqdm": "not-installed",
    "arch": "8.0.0"
  }
}


In [ ]:
CONFIG = {
    "R": 1000,
    "seed": 0,
    "n_list": [120, 240, 1200],
    "S_true_list": [0.0,0.25,0.5,0.75,1],
    "alpha": 0.05,
    "burn_B": 500,
    "t_df": 5,
    "garch_alpha": 0.05,
    "garch_beta": 0.90,
    "DSR_M": None,
    "max_workers": 4,
    "cache_dir": str((RUN_DIR / "cache").resolve()),
    "figures_dir": str((RUN_DIR / "figures").resolve()),
    "factors_start_date": "1926-07-01",
}

if CONFIG["t_df"] <= 4:
    raise ValueError("t_df must be > 4")
if CONFIG["garch_alpha"] + CONFIG["garch_beta"] >= 1:
    raise ValueError("garch_alpha + garch_beta must be < 1")

CACHE_DIR = Path(CONFIG["cache_dir"])
FIG_DIR = Path(CONFIG["figures_dir"])
CACHE_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(json.dumps(CONFIG, indent=2))

{
  "R": 1000,
  "seed": 0,
  "n_list": [
    120,
    240,
    1200
  ],
  "S_true_list": [
    0.0,
    0.5
  ],
  "alpha": 0.05,
  "burn_B": 500,
  "t_df": 5,
  "garch_alpha": 0.05,
  "garch_beta": 0.9,
  "DSR_M": null,
  "max_workers": 1,
  "cache_dir": "/workspaces/finance-data-download-test/notebooks/outputs/thesis_mvp/cache",
  "figures_dir": "/workspaces/finance-data-download-test/notebooks/outputs/thesis_mvp/figures",
  "factors_start_date": "1926-07-01"
}


In [23]:
# Project integration (ready-made functions only)
cwd = Path.cwd().resolve()
project_root = None
for p in [cwd, *cwd.parents]:
    if (p / "src" / "sharpe_mc.py").exists():
        project_root = p
        if str(p) not in sys.path:
            sys.path.insert(0, str(p))
        break

if project_root is None:
    raise RuntimeError("Could not find project root with src/sharpe_mc.py")

from src import sharpe_mc
from finance_data.french import load_us_research_factors_wide

print("Using project root:", project_root)
print("Using module:", sharpe_mc.__file__)

Using project root: /workspaces/finance-data-download-test
Using module: /workspaces/finance-data-download-test/src/sharpe_mc.py


In [24]:
# Small helper utilities (kept minimal)
OUTPUT_COLS = [
    "dgp", "n", "S_true", "method", "bias", "rmse", "coverage_95",
    "reject_rate_H0_S_le_0", "se_ratio", "psr_reject_rate", "dsr_reject_rate",
]


def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)


def save_df(df: pd.DataFrame, stem: str) -> list[Path]:
    paths = []
    pq = RUN_DIR / f"{stem}.parquet"
    csv = RUN_DIR / f"{stem}.csv"
    try:
        df.to_parquet(pq, index=False)
        paths.append(pq)
    except Exception:
        pass
    df.to_csv(csv, index=False)
    paths.append(csv)
    for p in paths:
        SAVED_ARTIFACTS.append(str(p.resolve()))
    return paths


def load_cached_df(stem: str) -> pd.DataFrame | None:
    pq = CACHE_DIR / f"{stem}.parquet"
    csv = CACHE_DIR / f"{stem}.csv"
    if pq.exists():
        try:
            return pd.read_parquet(pq)
        except Exception:
            pass
    if csv.exists():
        return pd.read_csv(csv)
    return None


def save_cached_df(df: pd.DataFrame, stem: str) -> Path:
    pq = CACHE_DIR / f"{stem}.parquet"
    csv = CACHE_DIR / f"{stem}.csv"
    try:
        df.to_parquet(pq, index=False)
        return pq
    except Exception:
        df.to_csv(csv, index=False)
        return csv


def save_fig(fig: plt.Figure, stem: str) -> None:
    png = FIG_DIR / f"{stem}.png"
    pdf = FIG_DIR / f"{stem}.pdf"
    fig.savefig(png, dpi=160, bbox_inches="tight")
    fig.savefig(pdf, bbox_inches="tight")
    SAVED_ARTIFACTS.extend([str(png.resolve()), str(pdf.resolve())])

In [25]:
# Part A: Monte Carlo calibration (using sharpe_mc.run_experiment directly)
set_seed(CONFIG["seed"])

cache_key = (
    f"partA_R{CONFIG['R']}_seed{CONFIG['seed']}_df{CONFIG['t_df']}_"
    f"ga{CONFIG['garch_alpha']}_gb{CONFIG['garch_beta']}_burn{CONFIG['burn_B']}_"
    f"alpha{CONFIG['alpha']}_n{','.join(map(str, CONFIG['n_list']))}_"
    f"S{','.join(map(str, CONFIG['S_true_list']))}"
)

partA_raw = load_cached_df(cache_key)
if partA_raw is None:
    rows = sharpe_mc.run_experiment(
        dgps=("iid_normal", "iid_t5", "garch11_t5"),
        n_grid=tuple(CONFIG["n_list"]),
        s_true_grid=tuple(CONFIG["S_true_list"]),
        reps=int(CONFIG["R"]),
        seed=int(CONFIG["seed"]),
        sigma=1.0,
        df=int(CONFIG["t_df"]),
        alpha=float(CONFIG["garch_alpha"]),
        beta=float(CONFIG["garch_beta"]),
        burn=int(CONFIG["burn_B"]),
        n_trials=int(CONFIG["DSR_M"]) if CONFIG["DSR_M"] is not None else sharpe_mc.DEFAULT_N_TRIALS,
        max_workers=int(CONFIG["max_workers"]),
        run_sanity=False,
    )
    partA_raw = pd.DataFrame(rows)
    save_cached_df(partA_raw, cache_key)

# Rename to thesis labels
results_partA = partA_raw.copy()
results_partA["dgp"] = results_partA["dgp"].replace({
    "iid_t5": "iid_t",
    "garch11_t5": "garch11_t",
})
results_partA["method"] = results_partA["method"].replace({
    "naive_asymptotic": "iid_normal_analytic",
    "robust_hac": "hac_newey_west",
})

# DSR: skip by default
if CONFIG["DSR_M"] is None:
    print("DSR skipped by default (set CONFIG['DSR_M'] to enable).")
    results_partA["dsr_reject_rate"] = np.nan
    dsr_mask = results_partA["method"].eq("dsr")
    results_partA.loc[dsr_mask, [
        "coverage_95", "reject_rate_H0_S_le_0", "se_ratio", "psr_reject_rate", "dsr_reject_rate"
    ]] = np.nan

results_partA = results_partA[OUTPUT_COLS].sort_values(["dgp", "n", "S_true", "method"]).reset_index(drop=True)
results_partA

DSR skipped by default (set CONFIG['DSR_M'] to enable).


,dgp,n,S_true,method,bias,rmse,coverage_95,reject_rate_H0_S_le_0,se_ratio,psr_reject_rate,dsr_reject_rate
0,garch11_t,120,0.000000,dsr,-0.001151,0.092662,NaN,NaN,NaN,NaN,NaN
1,garch11_t,120,0.000000,hac_newey_west,-0.001151,0.092662,0.926000,0.062000,0.951211,0.050000,NaN
2,garch11_t,120,0.000000,iid_normal_analytic,-0.001151,0.092662,0.945000,0.050000,0.986850,0.050000,NaN
3,garch11_t,120,0.000000,psr,-0.001151,0.092662,0.945000,0.050000,0.986850,0.050000,NaN
4,garch11_t,120,0.500000,dsr,0.022526,0.136823,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
67,iid_t,1200,0.000000,psr,0.001991,0.029550,0.937000,0.062000,0.978857,0.062000,NaN
68,iid_t,1200,0.500000,dsr,0.000445,0.034829,NaN,NaN,NaN,NaN,NaN
69,iid_t,1200,0.500000,hac_newey_west,0.000445,0.034829,0.947000,1.000000,0.969117,1.000000,NaN
70,iid_t,1200,0.500000,iid_normal_analytic,0.000445,0.034829,0.921000,1.000000,0.879041,1.000000,NaN


In [26]:
# Part A: save result table
paths = save_df(results_partA, "results_partA")
print("Saved Part A:")
for p in paths:
    print("-", p)

Saved Part A:
- /workspaces/finance-data-download-test/notebooks/outputs/thesis_mvp/results_partA.csv


In [27]:
# Part A sanity checks
checks = []

cov = results_partA[
    (results_partA["dgp"] == "iid_normal")
    & (results_partA["n"] == 1200)
    & (results_partA["S_true"] == 0.0)
    & (results_partA["method"] == "iid_normal_analytic")
]["coverage_95"]

if len(cov) == 1:
    value = float(cov.iloc[0])
    checks.append({"check": "iid normal coverage (n=1200, S=0)", "value": value, "pass": abs(value - 0.95) <= 0.02})
else:
    checks.append({"check": "iid normal coverage (n=1200, S=0)", "value": np.nan, "pass": False})

rng = np.random.default_rng(CONFIG["seed"] + 77)
t_big = sharpe_mc.simulate_iid_t5(n=200_000, mu=0.0, sigma=1.0, rng=rng, df=int(CONFIG["t_df"]))
var_t = float(np.var(t_big, ddof=1))
checks.append({"check": "standardized t variance ~ 1", "value": var_t, "pass": abs(var_t - 1.0) <= 0.08})

# GARCH sigma_t^2 positivity via direct recursion
nu = float(CONFIG["t_df"])
a = float(CONFIG["garch_alpha"])
b = float(CONFIG["garch_beta"])
omega = 1.0 - a - b
z = np.random.default_rng(CONFIG["seed"] + 88).standard_t(df=nu, size=5000) / np.sqrt(nu / (nu - 2.0))
h = np.empty_like(z)
eps_prev = 0.0
h_prev = 1.0
for i in range(len(z)):
    h_i = max(omega + a * eps_prev**2 + b * h_prev, 1e-14)
    eps_i = np.sqrt(h_i) * z[i]
    h[i] = h_i
    h_prev = h_i
    eps_prev = eps_i
checks.append({"check": "GARCH sigma_t^2 positive", "value": float(h.min()), "pass": bool((h > 0).all())})

sanity_df = pd.DataFrame(checks)
sanity_df

,check,value,pass
0,"iid normal coverage (n=1200, S=0)",0.954000,True
1,standardized t variance ~ 1,0.996854,True
2,GARCH sigma_t^2 positive,0.599336,True


In [28]:
# Part A plot: coverage vs n by method (one figure per DGP)
for dgp in ["iid_normal", "iid_t", "garch11_t"]:
    fig, axes = plt.subplots(1, len(CONFIG["S_true_list"]), figsize=(6 * len(CONFIG["S_true_list"]), 4), sharey=True)
    if len(CONFIG["S_true_list"]) == 1:
        axes = [axes]

    for ax, s_true in zip(axes, CONFIG["S_true_list"]):
        sub = results_partA[(results_partA["dgp"] == dgp) & (results_partA["S_true"] == float(s_true))]
        for method in ["iid_normal_analytic", "hac_newey_west", "psr", "dsr"]:
            m = sub[sub["method"] == method].sort_values("n")
            y = m["coverage_95"].to_numpy(dtype=float)
            if np.isfinite(y).any():
                ax.plot(m["n"], y, marker="o", label=method)
        ax.axhline(0.95, color="black", linestyle="--", linewidth=1)
        ax.set_title(f"{dgp} | S_true={s_true}")
        ax.set_xlabel("n")
        ax.set_ylabel("coverage_95")
        ax.set_xticks(CONFIG["n_list"])
        ax.set_ylim(0, 1)

    handles, labels = axes[0].get_legend_handles_labels()
    if handles:
        fig.legend(handles, labels, loc="upper center", ncol=min(4, len(labels)))
    fig.tight_layout()
    save_fig(fig, f"partA_coverage_{dgp}")
    plt.close(fig)

print("Saved Part A coverage figures")

Saved Part A coverage figures


In [29]:
# Part A plot: reject rate vs n by method (one figure per DGP)
for dgp in ["iid_normal", "iid_t", "garch11_t"]:
    fig, axes = plt.subplots(1, len(CONFIG["S_true_list"]), figsize=(6 * len(CONFIG["S_true_list"]), 4), sharey=True)
    if len(CONFIG["S_true_list"]) == 1:
        axes = [axes]

    for ax, s_true in zip(axes, CONFIG["S_true_list"]):
        sub = results_partA[(results_partA["dgp"] == dgp) & (results_partA["S_true"] == float(s_true))]
        for method in ["iid_normal_analytic", "hac_newey_west", "psr", "dsr"]:
            m = sub[sub["method"] == method].sort_values("n")
            y = m["reject_rate_H0_S_le_0"].to_numpy(dtype=float)
            if np.isfinite(y).any():
                ax.plot(m["n"], y, marker="o", label=method)
        ax.set_title(f"{dgp} | S_true={s_true}")
        ax.set_xlabel("n")
        ax.set_ylabel("reject_rate_H0_S_le_0")
        ax.set_xticks(CONFIG["n_list"])
        ax.set_ylim(0, 1)

    handles, labels = axes[0].get_legend_handles_labels()
    if handles:
        fig.legend(handles, labels, loc="upper center", ncol=min(4, len(labels)))
    fig.tight_layout()
    save_fig(fig, f"partA_reject_{dgp}")
    plt.close(fig)

print("Saved Part A reject-rate figures")

Saved Part A reject-rate figures


In [30]:
# Part B: load market return series from Fama-French and split train/holdout
partB_ready = False
partB_reason = ""
partB_n = 240

try:
    # Fetch Mkt-RF and RF back to 1926 (monthly, decimals)
    factors_wide, rf = load_us_research_factors_wide(start_date=CONFIG["factors_start_date"])
    factors_wide = factors_wide.rename(columns={"Mkt-RF": "Mkt_RF"}) if "Mkt-RF" in factors_wide.columns else factors_wide

    mkt_excess = factors_wide["Mkt_RF"].dropna()
    rf = rf.reindex(mkt_excess.index)
    mkt_total = mkt_excess + rf

    # Rescale return series to percent units.
    mkt_excess = mkt_excess * 100.0
    rf = rf * 100.0
    mkt_total = mkt_total * 100.0

    print("Coverage:")
    print("Start", mkt_excess.index.min().date(), "End", mkt_excess.index.max().date(), "N obs", mkt_excess.shape[0])

    # Part B uses the excess-return series for Sharpe inference
    series = mkt_excess

    T = len(series)
    train_len = int(round(0.70 * T))
    holdout_len = T - train_len

    train = series.iloc[:train_len]
    holdout = series.iloc[train_len:]

    n_windows = len(holdout) // partB_n
    if n_windows < 1:
        partB_reason = f"Skipped Part B: holdout has no disjoint n={partB_n} window"
        print(partB_reason)
    else:
        holdout_use = holdout.iloc[: n_windows * partB_n]
        obs_rows = []
        for j in range(n_windows):
            w = holdout_use.iloc[j * partB_n : (j + 1) * partB_n]
            s_hat, _, _ = sharpe_mc.sharpe_ratio(w.to_numpy(dtype=float))
            obs_rows.append({"window_id": j, "S_obs": float(s_hat)})
        obs_windows = pd.DataFrame(obs_rows)
        partB_ready = True
        print(f"Part B ready: T={T}, train={train_len}, holdout={holdout_len}, windows={n_windows}, n={partB_n}")
except Exception as e:
    partB_reason = f"Skipped Part B: failed to load Fama-French factors ({e})"
    print(partB_reason)

Coverage:
Start 1926-07-31 End 2025-09-30 N obs 1191
Part B ready: T=1191, train=834, holdout=357, windows=1, n=240


In [31]:
# Part B: fit 3 models, bootstrap Sharpe distribution, PIT diagnostics
partB_rows = []
fit_summary = {}

if not partB_ready:
    results_partB = pd.DataFrame(columns=[
        "model", "window_id", "S_obs", "pit_u", "ks_stat", "ks_pvalue", "pit_score", "n_windows", "n_boot"
    ])
    fit_summary = {"status": "skipped", "reason": partB_reason}
else:
    train_arr = train.to_numpy(dtype=float)
    Rb = int(CONFIG["R"])

    models = ["iid_normal", "iid_t_fixed_nu", "garch11_t_fixed_nu"]

    for model_name in models:
        cache_name = f"partB_boot_{model_name}_R{Rb}_n{partB_n}_seed{CONFIG['seed']}_nu{CONFIG['t_df']}"
        npy_path = CACHE_DIR / f"{cache_name}.npy"

        if npy_path.exists():
            sr_sorted = np.load(npy_path)
            cached = True
        else:
            cached = False
            if model_name == "iid_normal":
                model, res, params = sharpe_mc.fit_candidate(train_arr, "iid_normal")
                mu_hat = float(res.params["mu"])
                sigma_hat = float(np.sqrt(res.params["sigma2"]))
                fit_summary[model_name] = {"mu_hat": mu_hat, "sigma_hat": sigma_hat, "cached": cached}

                draws = mu_hat + sigma_hat * np.random.default_rng(CONFIG["seed"] + 1).standard_normal((Rb, partB_n))
                sr = draws.mean(axis=1) / draws.std(axis=1, ddof=1)

            elif model_name == "iid_t_fixed_nu":
                # Fixed nu: estimate mu/sigma from train, keep nu from config.
                mu_hat = float(np.mean(train_arr))
                sigma_hat = float(np.std(train_arr, ddof=1))
                nu = float(CONFIG["t_df"])
                fit_summary[model_name] = {"mu_hat": mu_hat, "sigma_hat": sigma_hat, "nu_fixed": nu, "cached": cached}

                rng = np.random.default_rng(CONFIG["seed"] + 2)
                z = rng.standard_t(df=nu, size=(Rb, partB_n)) / np.sqrt(nu / (nu - 2.0))
                draws = mu_hat + sigma_hat * z
                sr = draws.mean(axis=1) / draws.std(axis=1, ddof=1)

            else:  # garch11_t_fixed_nu
                model, res, params = sharpe_mc.fit_candidate(train_arr, "garch11_t")
                params_sim = np.asarray(params, dtype=float).copy()
                nu_hat_fit = float(params_sim[-1])
                params_sim[-1] = float(CONFIG["t_df"])  # closest fixed-nu approach
                init_var = float(max(np.var(train_arr, ddof=1), 1e-8))

                fit_summary[model_name] = {
                    "mu_hat": float(params_sim[0]),
                    "omega_hat": float(params_sim[1]),
                    "alpha_hat": float(params_sim[2]),
                    "beta_hat": float(params_sim[3]),
                    "nu_hat_fit": nu_hat_fit,
                    "nu_used": float(params_sim[-1]),
                    "note": "fixed nu not exposed by fitter; fitted params then replaced nu for simulation",
                    "cached": cached,
                }

                sr = np.empty(Rb, dtype=float)
                np.random.seed(CONFIG["seed"] + 3)
                for i in tqdm(range(Rb), desc=f"Part B {model_name}"):
                    sim = sharpe_mc.simulate_from_fit(
                        model,
                        params_sim,
                        n=partB_n,
                        burn=int(CONFIG["burn_B"]),
                        initial_value_vol=init_var,
                    )
                    s_hat, _, _ = sharpe_mc.sharpe_ratio(sim)
                    sr[i] = s_hat

            sr = sr[np.isfinite(sr)]
            sr_sorted = np.sort(sr)
            np.save(npy_path, sr_sorted)

        if model_name not in fit_summary:
            fit_summary[model_name] = {"cached": cached}

        u = np.searchsorted(sr_sorted, obs_windows["S_obs"].to_numpy(dtype=float), side="right") / len(sr_sorted)
        ks = stats.kstest(u, "uniform")
        score = float(np.mean(np.abs(u - 0.5)))

        for j, u_j in enumerate(u):
            partB_rows.append({
                "model": model_name,
                "window_id": int(obs_windows.iloc[j]["window_id"]),
                "S_obs": float(obs_windows.iloc[j]["S_obs"]),
                "pit_u": float(u_j),
                "ks_stat": float(ks.statistic),
                "ks_pvalue": float(ks.pvalue),
                "pit_score": score,
                "n_windows": int(len(u)),
                "n_boot": int(len(sr_sorted)),
            })

    results_partB = pd.DataFrame(partB_rows)

partB_paths = save_df(results_partB, "results_partB_pit")
fit_path = RUN_DIR / "model_fit_summary.json"
fit_path.write_text(json.dumps(fit_summary, indent=2), encoding="utf-8")
SAVED_ARTIFACTS.append(str(fit_path.resolve()))

print("Saved Part B:")
for p in partB_paths:
    print("-", p)
print("-", fit_path)

/usr/local/python/3.12.1/lib/python3.12/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.003138. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


Saved Part B:
- /workspaces/finance-data-download-test/notebooks/outputs/thesis_mvp/results_partB_pit.csv
- /workspaces/finance-data-download-test/notebooks/outputs/thesis_mvp/model_fit_summary.json


In [32]:
# Part B diagnostics view
if results_partB.empty:
    print(partB_reason if partB_reason else "Part B skipped")
else:
    diag = results_partB[["model", "ks_stat", "ks_pvalue", "pit_score", "n_windows", "n_boot"]].drop_duplicates()
    diag

In [33]:
# Part B plot: PIT histogram per model
if results_partB.empty:
    print("No Part B plot (results empty)")
else:
    for model_name, g in results_partB.groupby("model"):
        fig, ax = plt.subplots(figsize=(6, 4))
        u = g["pit_u"].to_numpy(dtype=float)
        ax.hist(u, bins=np.linspace(0, 1, 11), density=True, alpha=0.75, edgecolor="black")
        ax.axhline(1.0, color="black", linestyle="--", linewidth=1)
        ax.set_xlim(0, 1)
        ax.set_xlabel("PIT u")
        ax.set_ylabel("density")
        ax.set_title(f"PIT histogram | {model_name}")
        fig.tight_layout()
        save_fig(fig, f"partB_pit_hist_{model_name}")
        plt.close(fig)
    print("Saved Part B PIT histogram figures")

Saved Part B PIT histogram figures


In [34]:
# Part B plot: ECDF(u) vs uniform per model
if results_partB.empty:
    print("No Part B ECDF plot (results empty)")
else:
    for model_name, g in results_partB.groupby("model"):
        u = np.sort(g["pit_u"].to_numpy(dtype=float))
        y = np.arange(1, len(u) + 1) / len(u)

        fig, ax = plt.subplots(figsize=(6, 4))
        ax.step(u, y, where="post", label="ECDF(u)")
        ax.plot([0, 1], [0, 1], linestyle="--", color="black", label="Uniform")
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_xlabel("u")
        ax.set_ylabel("ECDF")
        ax.set_title(f"PIT ECDF | {model_name}")
        ax.legend(loc="lower right")
        fig.tight_layout()
        save_fig(fig, f"partB_pit_ecdf_{model_name}")
        plt.close(fig)
    print("Saved Part B ECDF figures")

Saved Part B ECDF figures


In [35]:
# Final summary
print("Part A head:")
print(results_partA.head())

print("\nPart B head:")
print(results_partB.head())

print("\nSaved artifacts:")
for p in sorted(set(SAVED_ARTIFACTS)):
    print("-", p)


Part A head:
         dgp    n   S_true               method      bias     rmse  coverage_95  reject_rate_H0_S_le_0  se_ratio  psr_reject_rate  dsr_reject_rate
0  garch11_t  120 0.000000                  dsr -0.001151 0.092662          NaN                    NaN       NaN              NaN              NaN
1  garch11_t  120 0.000000       hac_newey_west -0.001151 0.092662     0.926000               0.062000  0.951211         0.050000              NaN
2  garch11_t  120 0.000000  iid_normal_analytic -0.001151 0.092662     0.945000               0.050000  0.986850         0.050000              NaN
3  garch11_t  120 0.000000                  psr -0.001151 0.092662     0.945000               0.050000  0.986850         0.050000              NaN
4  garch11_t  120 0.500000                  dsr  0.022526 0.136823          NaN                    NaN       NaN              NaN              NaN

Part B head:
                model  window_id    S_obs    pit_u  ks_stat  ks_pvalue  pit_score  n_window